
## The assignTaxonomy(...) function

The primary goal of [the dada2 package](http://benjjneb.github.io/dada2/) is to infer the exact sequence variants in amplicon sequenced samples and their associated abundances. However, a common companion step, especially when sequencing taxonomic marker genes such as 16S or 18S rRNA, is to assign taxonomies to the inferred sequence variants (or OTUs if using an OTU method).

In order to facilitate this, the dada2 package provides the `assignTaxonomy(...)` function, which implements the RDP classifier method described in [Wang et al. 2007](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC1950982/). In short, the kmer profile of the sequences to be classified are compared against the kmer profiles of all sequences in a training set of sequences with assigned taxonomies. The reference sequence with the most similar profile is used to assign taxonomy to the query sequence, and then a bootstrapping approach is used to assess the confidence of the taxonomic assignment.



In [ ]:
library(dada2); packageVersion("dada2")


We'll go through a short example to demonstrate the method in action, but first we consider the issue of the training dataset.

## Taxonomic training sets

Gzip-ed version of [the **dada2-compatible fastas for RDP, Silva and GreenGenes can be downloaded here**](https://www.dropbox.com/sh/mfcivbudmc21cqt/AAB1l-AUM5uKvjrR33ct-cTXa?dl=0). Please be aware that the Silva uses a [dual licensing model for academia and commercial users](https://www.arb-silva.de/silva-license-information/).

The `assignTaxonomy(...)` function expects the training data to be provided in the form of a fasta file (or compressed fasta file) in which the taxonomy corresponding to each sequence is encoded in the id line in the following fashion:

```
>Level1;Level2;Level3;Level4;Level5;Level6;
ACCTAGAAAGTCGTAGATCGAAGTTGAAGCATCGCCCGATGATCGTCTGAAGCTGTAGCATGAGTCGATTTTCACATTCAGGGATACCATAGGATAC
>Level1;Level2;Level3;Level4;Level5;Level6;
CGCTAGAAAGTCGTAGAAGGCTCGGAGGTTTGAAGCATCGCCCGATGGGATCTCGTTGCTGTAGCATGAGTACGGACATTCAGGGATCATAGGATAC
```

In order to simplify the most common use case, we have provided appropriately formatted training fasta files for the three most common datasets used for 16S classification: the [RDP training set 14](http://rdp.cme.msu.edu/), the [Silva v123 database](http://www.arb-silva.de/documentation/release-123/), and [the Green Genes 13.8 release](ftp://greengenes.microbio.me/greengenes_release/gg_13_8_otus/).

In addition to thanking the folks at RDP, Silva and GreenGenes for making these datasets available, we also want to thank Pat Schloss and the [mothur](http://www.mothur.org) team for making cleaner versions of the Silva and RDP training set available. To be specific, we created the dada2-compatible training fastas from [the mothur-compatible Silva.nr_v123 files](http://www.mothur.org/wiki/Silva_reference_files) ([described here](http://blog.mothur.org/2015/12/03/SILVA-v123-reference-files/), and [license here](https://www.arb-silva.de/silva-license-information/)), the [mothur-compatible 16S rRNA reference (RDP)](http://mothur.org/wiki/RDP_reference_files) ([described here](http://blog.mothur.org/2015/05/27/RDP-v14-reference_files/)), and [the GreenGenes 13.8 OTUs clustered at 97\%](ftp://greengenes.microbio.me/greengenes_release/gg_13_8_otus/).

*In order to follow along with the examples below, [download](https://www.dropbox.com/sh/mfcivbudmc21cqt/AAB1l-AUM5uKvjrR33ct-cTXa?dl=0) the `rdp_train_set_14.fa.gz` file to an appropriate location on your local machine.*

## Taxonomic assignment in action

For now we will consider a simple example in which we want to classify a handful of 16S sequences from the v4 region against the RDP training set. Our sequences of interest:


In [ ]:
seqs <- c(
"ACGGAGGGTCCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAATTGCATTTCGAACTGGCGAACTAGAGTCTTGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGA",
"ACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTGCGTAGGCTGTTTCGTAAGTCGTGTGTGAAAGGCGCGGGCTCAACCCGCGGACGGCACATGATACTGCGAGACTAGAGTAATGGAGGGGGAACCGGAATTCTCGGTGTAGCAGTGAAATGCGTAGATATCGAGAGGA",
"ACGGAGGGTGCAAGCGTTAATCGGAATCACTGGGCGTAAAGCGCACGTAGGCTGTTATGTAAGTCAGGGGTGAAAGCCCACGGCTCAACCGTGGAACTGCCCTTGATACTGCACGACTCGAATCCGGGAGAGGGTGGCGGAATTCCAGGTGTAGGAGTGAAATCCGTAGATATCTGGAGGA",
"ACGTAGGTCCCGAACGTTGCGCGAATTTACTGGGCGTAAAGGGTCCGTAGGCGGTTTAGCAAGTGGTTGGTGAAATTTCACGGCTCAACCGTGAAACTGCCTTCCAAACTGCTAAACTTGAGGCAGGGAGAGGTCGGCGGAATTCCCGGTGTAGCGGTGAAATGCGTAGATATCGGGAGGA",
"ACGGAGGGGGTTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGTACGTAGGCGGATTGGAAAGTTGGGGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTCCAAAACTATCAGTCTAGAGTTCGAGAGAGGTGAGTGGAATTCCAAGTGTAGAGGTGAAATTCGTAGATATTTGGAGGA",
"ACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGTGGACAGTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGGTGTAGCGGTGAAATGCTTAGATATCACGAAGA",
"ACGTAGGGCGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGAGCTCGTAGGCGGCTTGTCGCGTCGACTGTGAAAACCCGTGGCTCAACTGCGGGCTTGCAGTCGATACGGGCAGGCTAGAGTTCGGTAGGGGAGACTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGA",
"ACGGAGGGTGCAAGCGTTAATCGGAATGACTGGGCGTAAAGCGCACGCAGGCGGTCTGTTAAGTTGGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATTCAAAACTGACAGGCTAGAGTCTCGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGA",
"ACCGACGGCCCGAGTGGTGGCCACTCTTATTGGGCCTAAAGCGTCCGTAGCCGGTCCAGTAAGTCCCTGTTTAAATCCTGCGGCTTAACCGCAGGACTGGCAGGGATACTGCTGGACTTGGGACCGGGAGAGGACAAGGGTACTTCAGGGGTAGCGGTGAAATGTGTTGATCCTTGAAGGA",
"ACGTAGGTCCCGAACGTTGCGCGAAATTACTGGGCGTAAAGGGTCCGTAGGCGGTCTGGTAAGTGGAAGGTGAAAGCCCACGGCTCAACCGTGGAATTGCCTTCCAAACTGCTGGACTTGAGGGCGGAAGAGGTCGGCGGAATTCCCGGTGTAGCGGTGAAATGCGTAGATATCGGGAGGA")


We now run the `assignTaxonomy(...)` function against the RDP training set. *To run the following on your local machine, modify the filename in the second argument to point to where the `rdp_train_set_14.fa.gz` file sits on your computer.*



In [ ]:
set.seed(100) # Initialize random number generator for reproducibility
taxa <- assignTaxonomy(seqs, "rdp_train_set_14.fa.gz")
unname(taxa)


And as easy as that we have our taxonomic assignments! The return value of `assignTaxonomy(...)` is a character matrix, with each row corresponding to an input sequence, and each column corresponding to a taxonomic level. For the RDP training set, the levels are Kingdom, Phylum, Class, Order, Family, Genus, and we see that each sequence has been classified all the way to genus.

There is one important parameter to consider when running `assignTaxonomy(...)`, and that is the `minBoot` parameter that sets the minimum bootstrapping confidence required to return a taxonomic classification. The orginal paper recommended a threshhold of 50 for sequences of 250nts or less (as these are) but a threshhold of 80 generally. Let's see what happens when we classify at that higher threshhold:


In [ ]:
set.seed(100) # Initialize random number generator for reproducibility
taxa.80 <- assignTaxonomy(seqs, "rdp_train_set_14.fa.gz", minBoot=80)
unname(taxa.80)


At this more stringent threshhold, sequence 5 and 7 are not classified at the genus level, but instead an NA is returned, indicating that less than `minBoot=80` of the 100 bootstraps returned the same genus for those sequences.

For a bit more practice, consider assigning these sequences against the Silva and GreenGenes training fastas as well, and exploring the consistency of taxonomic assignment across the reference databases!
